# 数据预处理



首先导入数据处理包。

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

**加载并分割数据**

In [2]:
train=pd.read_csv('train.csv')
images=train.iloc[:,1:]
labels_flat=train.iloc[:,0]

**对输入进行处理**

In [3]:
images=images.astype(np.float)
#归一化处理
images=np.multiply(images,1.0/255.0)
print('输入数据的数量：(%g,%g)'% images.shape)
image_size=images.shape[1]
print('输入数据的维度=>%g'% image_size)
image_width = image_height=np.ceil(np.sqrt(image_size))
print('图片的长=>%g\n图片的高=>%g'%(image_width,image_height))
labels_count=np.unique(labels_flat).shape[0]
print('结果的种类=>%g'%labels_count)

输入数据的数量：(42000,784)
输入数据的维度=>784
图片的长=>28
图片的高=>28
结果的种类=>10


**进行One-hot编码**

In [4]:
# 进行One-hot编码
def dense_to_one_hot(labels_dense,num_classes):
    num_labels=labels_dense.shape[0]
    # 计算每一行所需要的偏移量
    index_offset=np.arange(num_labels)*num_classes
    labels_one_hot=np.zeros((num_labels,num_classes))
    # 把一个n*n的矩阵看做是一个1*n²的行向量，进行赋值
    labels_one_hot.flat[index_offset+labels_dense.ravel()]=1
    # 得到one-hot
    return labels_one_hot

labels=dense_to_one_hot(labels_flat,labels_count)
labels=labels.astype(np.uint8)
print('结果的数量：(%g,%g)'%labels.shape)

结果的数量：(42000,10)


In [5]:
x=tf.placeholder('float',shape=[None,image_size])
y=tf.placeholder('float', shape=[None, labels_count])

**把输入数据划分为训练集和验证集**

In [6]:
#2000个数据作为验证集、其他40000个做训练集
VALIDATION_SIZE=2000

validation_images=images[:VALIDATION_SIZE]
validation_labels=labels[:VALIDATION_SIZE]

train_images=images[VALIDATION_SIZE:]
train_labels=labels[VALIDATION_SIZE:]

**对训练集进行分批**

In [7]:
batch_size=100
n_batch=len(train_images)/batch_size

# 搭建神经网络模型

我们的神经网络的超参数：

    1. 激活函数：softmax
    2. 损失函数：交叉熵
    3. 优化方式：梯度下降
    4. 输入层：784个节点，对应每个像素点。
    5. 隐层：只有一个隐层，10个节点，同时作为输出层。
    6. 输出层：就是隐层。

In [8]:
# 这个weights表示第一隐层只有10个节点
weights=tf.Variable(tf.zeros([784,10]))
# biases,偏差，相当于y=ax+b中的b
biases=tf.Variable(tf.zeros([10]))
# result,定义第一隐层的输出
result=tf.matmul(x,weights)+biases
# prediction，激活函数，对线性计算结果添加非线性内容。
prediction=tf.nn.softmax(result)
#创建损失函数了，以交叉熵的平均值来衡量
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
#用梯度下降法优化参数
train_step=tf.train.GradientDescentOptimizer(0.1).minimize(loss)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



# 初始化变量并运行

我们的神经网络的超参数：

    1. 初始化变量
    2. 计算准确度
    3. 在session中训练  

In [9]:
#初始化变量
init=tf.global_variables_initializer()
#计算准确度
correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [10]:
global_step=tf.Variable(0,name='global_step',trainable=False)
saver=tf.train.Saver()

In [11]:
#在session中进行训练
with tf.Session() as sess:
    # 初始化变量
    sess.run(init)
    
    # 循环50轮
    for epoch in range(50):
        for batch in range(int(n_batch)):
            # 按照batch取出数据
            batch_x=train_images[batch*batch_size:(batch+1)*batch_size]
            batch_y=train_labels[batch*batch_size:(batch+1)*batch_size]
            # 开始训练
            sess.run(train_step,feed_dict={x:batch_x,y:batch_y})
        # 每一轮训练计算结果
        accuracy_n=sess.run(accuracy,feed_dict={x:validation_images,y:validation_labels})
        # 打印结果
        print("第"+str(epoch+1)+"轮，准确度为："+str(accuracy_n))
        # 保存训练模型
        global_step.assign(epoch).eval()
        saver.save(sess,"save/model.ckpt",global_step=global_step)

第1轮，准确度为：0.7925
第2轮，准确度为：0.808
第3轮，准确度为：0.8155
第4轮，准确度为：0.8535
第5轮，准确度为：0.876
第6轮，准确度为：0.8875
Instructions for updating:
Use standard file APIs to delete files with this prefix.
第7轮，准确度为：0.8915
第8轮，准确度为：0.895
第9轮，准确度为：0.897
第10轮，准确度为：0.897
第11轮，准确度为：0.899
第12轮，准确度为：0.9005
第13轮，准确度为：0.901
第14轮，准确度为：0.9035
第15轮，准确度为：0.9065
第16轮，准确度为：0.9075
第17轮，准确度为：0.908
第18轮，准确度为：0.909
第19轮，准确度为：0.9105
第20轮，准确度为：0.9105
第21轮，准确度为：0.911
第22轮，准确度为：0.911
第23轮，准确度为：0.9115
第24轮，准确度为：0.912
第25轮，准确度为：0.9125
第26轮，准确度为：0.9135
第27轮，准确度为：0.9135
第28轮，准确度为：0.9135
第29轮，准确度为：0.9135
第30轮，准确度为：0.914
第31轮，准确度为：0.9145
第32轮，准确度为：0.915
第33轮，准确度为：0.915
第34轮，准确度为：0.915
第35轮，准确度为：0.916
第36轮，准确度为：0.917
第37轮，准确度为：0.918
第38轮，准确度为：0.919
第39轮，准确度为：0.919
第40轮，准确度为：0.9195
第41轮，准确度为：0.9195
第42轮，准确度为：0.9195
第43轮，准确度为：0.919
第44轮，准确度为：0.9195
第45轮，准确度为：0.9185
第46轮，准确度为：0.919
第47轮，准确度为：0.919
第48轮，准确度为：0.919
第49轮，准确度为：0.919
第50轮，准确度为：0.9195


In [12]:
#现在可以开始对测试集进行预处理了
test=pd.read_csv('test.csv',index_col=False)
test_x=test.iloc[:,1:].values
test_x=test_x.astype(np.float)
test_x=np.multiply(test_x,1.0/255.0)

In [13]:
#我们用我们训练好的模型开始预测，把结果写入到Digit1.csv
with tf.Session() as sess:
    sess.run(init)
    
    saver.restore(sess,'save/model.ckpt-49')
    
    test_x=np.array(test,dtype=np.float32)
    
    y_predict=prediction.eval(feed_dict={x:test_x[1:100,:]})
    y_predict_all=list()
    
    for i in np.arange(100,28001,100):
        
        y_predict=prediction.eval(feed_dict={x:test_x[i-100:i,:]})
        
        test_pred=np.argmax(y_predict,axis=1)
        
        y_predict_all=np.append(y_predict_all,test_pred)
    
    submission=pd.DataFrame(data={'ImageId':range(1,28001),'Label':np.int32(y_predict_all)})
    
    submission.to_csv('Digit1.csv',index=False)

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from save/model.ckpt-49
